In [0]:
import numpy as np
# import PyPDF2
from nltk.corpus import stopwords
import re

In [0]:
# import textract

In [0]:
# !pip3 install PyPDF2

In [0]:
# !pip3 install textract

### Pdf extact text

In [0]:
pdf_obj = open("One_Indian_Girl_-_Chetan_Bhagat-Redicals.pdf","rb")

In [0]:
reader = PyPDF2.PdfFileReader(pdf_obj)

In [0]:
No_of_pg = reader.numPages
print(No_of_pg)
pg_obj = reader.getPage(50)
pg_obj.extractText()

232


''

In [0]:
# text = ''
# for i in range(232):
# #     pg_obj = reader.getPage(i)
#     text = text + pg_obj.extractText()
# #     print(text)
# text

## PreProcessng Text File 

In [0]:
pdf_obj.close()

In [0]:
f = open("./One_Indian_Girl_-_Chetan_Bhagat-Redicals.txt")

In [0]:
text = f.read()

In [0]:
f.close()

In [15]:
print(len(text))
print(text[:500])
type(text)

464927
 

ONE
INDIAN
GIRL

Chetan Bhagat is the author of six bestselling novels—Fl've Point Someone (2004), One Night @ the
Call Center (2005), The 3Mislakes ofMy Life (2008), 2 States (2009), Revolution 2020 (2011) and
HalfGirlfriend (2014)—which have sold over ten million copies and have been translated into over
a dozen languages worldwide.

In 2008, The New York Times called him ‘the biggest—selling author in India’s history’, a
position he has maintained to date. Almost all his books have been ad


str

In [0]:
# text = text.lower()
# text = re.sub("[^A-Za-z]+"," ",text)#^[a-zA-Z] means any a-z or A-Z at the start of a line
# text = text.split()   #[^a-zA-Z] means any character that IS NOT a-z OR A-Z

## Normalise Document

In [20]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")
tknizer = nltk.WordPunctTokenizer()
stop_wrd = stopwords.words("english")
corpus = nltk.sent_tokenize(text)
print(len(corpus))
corpus[890]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
11245


'“Hi, Brij esh.'

In [0]:
def normalise_document(doc):
    doc = re.sub(r"^a-zA-Z\s","",doc,re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    
    words = tknizer.tokenize(doc)
#     print(words[5:10])
    fil_words = [w for w in words if w not in stop_wrd]
    
    doc = " ".join(fil_words)
    return doc

In [0]:
norm_doc = np.vectorize(normalise_document) #it only transfer your functino in numpy for broad casting purpose
doc  = norm_doc(corpus)

In [23]:
doc = [sent for sent in doc if len(sent.split())>2]
print(type(doc))
print(len(doc))
doc[890]

<class 'list'>
10299


'phone buzzed .'

## Corpus Vocubulary

In [24]:
from keras.preprocessing import text
from keras.utils import np_utils
from keras.preprocessing import sequence

Using TensorFlow backend.


In [0]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(doc)
word2id = tokenizer.word_index

In [26]:
print(len(word2id.keys()))

6556


In [27]:
text.text_to_word_sequence(doc[520])

['set',
 'routines',
 'protocols',
 'tools',
 'building',
 'software',
 'applications']

In [28]:
word2id["pad"] = 0
id2word = dict([(v,k) for k , v in word2id.items()])
wids = [[word2id[w] for w in text.text_to_word_sequence(d)] for d in doc]

vocab_size = len(word2id)
emb_size = 100
win_size = 1

print(vocab_size)
print(list(word2id.items())[5])
print(list(id2word.items())[5])
print(list(wids)[8])

6556
('like', 6)
(6, 'like')
[1028, 80, 478, 1850, 368, 2391, 1851, 350, 858, 1150, 1852, 3491, 142, 602, 2392, 23, 2387]


## generate context word

In [0]:
def generate_context_pair(corpus , wind_size , vocab_size ):
    for words in corpus:
        for idx , word in enumerate(words):
            cnt_word = []
            label_word = []
            start = idx - wind_size
            end = idx +wind_size + 1
            
            cnt_word.append([words[i] for i in range(start,end) if i!= idx if 0 <= i < len(words)])
            
            label_word.append(word)
            
            x = sequence.pad_sequences(cnt_word,maxlen = wind_size * 2)
            y = np_utils.to_categorical(label_word,vocab_size)
            yield(x , y)

In [30]:
# x = generate_context_pair(wids,2,vocab_size=vocab_size)
cnt = 0
for x ,y in generate_context_pair(wids,2,vocab_size):
    if 0 not in x[0]:
        print("*"*58)
        print("shape of training data --> ",x.shape, y.shape)
#         print(i[0]
        
        id_ = np.argwhere(y[0])
        print("Return the idx of non zero value -",id_," <=> ",id_[0]," <=> ",id_[0][0])
        print([id2word[id] for id in x[0]],"-->",id2word[id_[0][0]])
    cnt+=1
    if cnt == 5:
        break

**********************************************************
shape of training data -->  (1, 4) (1, 6556)
Return the idx of non zero value - [[60]]  <=>  [60]  <=>  60
['one', 'indian', 'chetan', 'bhagat'] --> girl
**********************************************************
shape of training data -->  (1, 4) (1, 6556)
Return the idx of non zero value - [[1028]]  <=>  [1028]  <=>  1028
['indian', 'girl', 'bhagat', 'author'] --> chetan
**********************************************************
shape of training data -->  (1, 4) (1, 6556)
Return the idx of non zero value - [[945]]  <=>  [945]  <=>  945
['girl', 'chetan', 'author', 'six'] --> bhagat


## np.sum vs k.sum2

In [0]:
def compareK_np_sum(x):
        print()
        print("^"*60)
        print("X --> ",x)
        print("sum along axis 0 (np) -->",np.sum(x,axis = 0,keepdims = True))
        sum_ = K.sum(x,axis = 0 ,keepdims = True)
        print(sum_)
        print("sum along axis 1 (np) -->",np.sum(x,axis = 1,keepdims = True))
        sum__= K.sum(x,axis = 1 ,keepdims = True)
        print(sum__)
        print("^"*60)
        del sum_ ,sum__
        print()

In [34]:
x_ = np.array([[2,3],[4,3]])
print(x_.shape)
compareK_np_sum(x_)

(2, 2)

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
X -->  [[2 3]
 [4 3]]
sum along axis 0 (np) --> [[6 6]]
Tensor("Sum:0", shape=(1, 2), dtype=int64)
sum along axis 1 (np) --> [[5]
 [7]]
Tensor("Sum_1:0", shape=(2, 1), dtype=int64)
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^



## Model Architecture

In [0]:
import keras.backend as K
from keras.layers import Dense , Embedding , Lambda
from keras.models import Sequential

In [35]:
cbow = Sequential()

cbow.add(Embedding(input_dim = vocab_size,output_dim = emb_size ,input_length = win_size * 2))
cbow.add(Lambda(lambda x : K.mean(x , axis = 1)))
cbow.add(Dense(vocab_size,activation = "softmax"))

print(cbow.summary())

W0805 15:43:02.174518 139752826812288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0805 15:43:02.179760 139752826812288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0805 15:43:02.184044 139752826812288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2, 100)            655600    
_________________________________________________________________
lambda_1 (Lambda)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6556)              662156    
Total params: 1,317,756
Trainable params: 1,317,756
Non-trainable params: 0
_________________________________________________________________
None


## Train Model

In [0]:
cbow.compile(loss = "categorical_crossentropy",optimizer = "rmsprop")

In [42]:
for epoch in range(1,5):
    loss = 0
    cnt = 0
    for x,y in generate_context_pair(wids,win_size,vocab_size):
        cnt+=1
#         print(loss)
        loss += cbow.train_on_batch(x,y)
        if cnt > vocab_size:
           break
        
    print("epoch",epoch , "\tloss",loss)
    print()

epoch 1 	loss 63401.61876747315

epoch 2 	loss 62374.24032921763

epoch 3 	loss 62288.088334573025

epoch 4 	loss 62185.4817619813



## Get Word **weights**

In [47]:
weights = cbow.get_weights()[0]
weights = weights[1:]

print(weights.shape)

import pandas as pd
wrd_emb = pd.DataFrame(weights , index = list(id2word.values())[1:])
wrd_emb.head(6)

(6555, 100)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
‘,-0.022720,0.135306,-0.053282,0.121484,0.598269,-0.056987,0.135193,0.099447,0.057409,-0.056849,0.090632,0.631652,0.156660,-0.098575,0.101455,0.136497,0.350908,-0.343832,-0.361281,-0.286316,0.467748,0.072726,-0.249610,0.344174,-0.070469,-0.164689,0.223758,0.209016,-0.149447,0.061665,-0.026548,-0.146506,-0.395806,0.540297,-0.484221,-0.011347,0.140885,0.241479,0.160085,0.046961,...,0.688237,0.025743,0.115009,0.820390,-0.261331,0.343398,-0.112369,-0.178766,0.230802,-0.004720,-0.234477,-0.279297,0.025190,-0.256675,0.070261,0.291632,-0.576976,-0.037627,0.116572,-0.460719,0.110156,-0.084440,0.047169,0.294285,0.075878,0.209045,0.023149,-0.018986,-0.316334,-0.201368,0.119469,0.009170,-0.232622,0.127752,-0.153808,-0.166507,-0.200365,-0.286396,-0.145619,0.104314
said,-0.071047,-0.216564,-0.271113,-0.234575,0.478464,-0.514455,0.305571,0.184223,-0.013023,-0.163950,-0.081322,0.082421,0.345518,0.331016,0.221741,-0.131270,0.256285,-0.294370,-0.205948,-0.256354,0.244627,0.019041,-0.554600,0.064867,0.480522,0.663244,0.594983,-0.398030,-0.130733,-0.158510,-0.038736,0.242217,-0.184277,-0.194513,0.376491,-0.137999,-0.087950,-0.029856,-0.052758,0.957080,...,-0.019394,-0.296147,0.113076,0.122716,-0.292727,-0.431050,-0.333651,0.268257,-0.096615,0.338092,0.177870,0.091716,0.150719,0.235350,-0.237254,0.608982,0.064278,0.154034,0.052718,0.163398,0.431844,0.407980,-0.172432,0.107723,-0.380203,-0.228786,-0.041154,-0.134404,0.001685,0.113732,-0.246785,-0.325208,-0.185590,0.284324,0.212001,-0.247207,0.209432,-0.224090,-0.368581,0.556418
neel,-0.254439,-0.317121,-0.025290,-0.351523,0.606926,0.257070,0.744285,-0.649370,0.328571,-0.688320,-0.298092,-0.182848,0.338710,-0.525844,-0.503951,0.530372,0.422124,0.890085,0.115014,-0.340959,-0.242414,0.930674,-0.265488,0.054730,-0.271078,-0.401899,-0.256024,-0.149805,0.311821,0.091444,-0.599095,-0.129380,-0.106493,-0.274409,0.929977,0.194982,0.232767,0.065394,0.766891,0.115428,...,0.046601,-0.461798,0.016357,0.556118,0.291537,-0.650683,-0.128858,0.103960,-0.195352,-0.900191,0.038515,-0.162373,0.144938,0.506582,0.873350,-0.423681,-0.076523,0.347845,-0.001649,0.086025,-0.447499,0.236199,0.146426,0.151752,0.944804,-0.138569,0.108387,0.435776,0.016523,0.056811,0.072710,1.044848,0.143544,0.117475,-0.389716,-0.332104,-0.624271,0.273954,-0.283569,-0.725304
debu,0.079101,-0.191456,-0.075812,-0.109067,0.123679,0.139692,-0.046820,-0.206601,-0.167150,-0.116812,0.029807,-0.009269,0.218260,-0.221215,-0.043163,0.243244,0.088473,0.128445,-0.028076,0.334555,-0.025995,0.192249,-0.202985,-0.084837,0.138314,-0.029930,-0.292446,-0.024085,-0.145456,0.072794,0.060098,-0.111943,0.230706,-0.378851,-0.045952,0.000528,0.115890,-0.024007,0.287755,0.051467,...,0.268751,-0.040652,-0.025268,-0.038200,-0.055769,-0.010843,-0.039047,-0.114481,-0.056893,-0.354773,-0.111986,-0.015663,0.454939,0.046737,0.177847,0.138950,0.239357,0.075996,0.063850,0.425224,-0.076774,0.081448,0.121748,-0.071379,0.157690,-0.013687,-0.047884,-0.111301,-0.024795,-0.102052,0.175763,-0.130294,0.032943,0.051100,-0.163464,0.172571,-0.175847,0.276834,0.033702,-0.180989
like,0.024025,-0.111846,-0.180834,-0.190905,0.096434,0.004204,-0.154967,-0.233875,-0.082517,-0.063315,0.028798,0.086468,0.156173,-0.258505,0.001367,0.285562,0.013597,0.143057,-0.116560,0.242926,0.037774,0.278875,-0.188272,-0.079485,0.250280,-0.016250,-0.001199,-0.172533,0.064901,0.129417,-0.041981,0.012573,-0.227190,-0.159310,-0.118246,0.115272,-0.000028,-0.064672,0.214774,-0.017278,...,0.210811,-0.191131,0.110307,0.106230,-0.006660,-0.188051,0.017953,-0.096126,0.010511,-0.325529,0.025202,-0.065541,-0.025933,0.163005,0.342708,0.092819,-0.039346,0.046283,0.103579,0.028244,-0.107619,0.150068,-0.219627,0.059553,0.192227,-0.011362,-0.038693,-0.031027,-0.255753,-0.028828,0.019747,0.256152,

## Simililarity of word

In [50]:
from sklearn.metrics.pairwise import cosine_similarity

c_matrix = cosine_similarity(weights)
print(c_matrix.shape)

similar_words = {search_term: [id2word[idx] for idx in c_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['facebook', 'california', 'phone', 'bus', 'chetan', 'airport', 'boys','man']}

similar_words

(6555, 6555)


{'airport': ['14', 'shudder', 'writer', 'logi', 'goof'],
 'boys': ['fl', 'goof', 'gary', 'negotiate', 'torch'],
 'bus': ['fairy', 'disruption', 'ﬁimbling', 'hindsight', 'americans'],
 'california': ['lcthl', 'singles', 'gents', 'split', 'l'],
 'chetan': ['jiju', 'gary', 'weds', 'decreased', 'advertisement'],
 'facebook': ['feared', 'peg', 'goof', 'gary', 'vivita'],
 'man': ['consume', 'fl', 'torch', 'temptation', 'exaggerating'],
 'phone': ['torch', 'bearer', 'heaved', 'various', 'prizes']}